In [21]:
import os
import io
import time
import pickle
import pandas as pd
import numpy as np

import cv2
from skimage import transform
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

#MNIST 1x28x28

In [46]:
dataset_root = r"C:\Users\Leo's PC\PycharmProjects\PD\MNIST"
data_transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

MNIST_train = torchvision.datasets.MNIST(dataset_root, train=True, transform=data_transform, target_transform=None, download=True)
MNIST_val = torchvision.datasets.MNIST(dataset_root, train=False, transform=data_transform, target_transform=None, download=True)

train_loader = DataLoader(dataset=MNIST_train, batch_size=100,shuffle=True) # 600 batches
val_loader = DataLoader(dataset=MNIST_val, batch_size=100, shuffle=False) # 100 batches

In [47]:
class Avg_Combine(nn.Module):
    def __init__(self):
        super(Avg_Combine, self).__init__()
        
    def forward(self, a, b):
        if a.shape[2] < b.shape[2]:
            factor = b.shape[2] / a.shape[2]
            a = functional.interpolate(a, scale_factor=factor)
        elif a.shape[2] > b.shape[2]:
            factor = a.shape[2] / b.shape[2]
            b = functional.interpolate(b, scale_factor=factor)
        
        idx0 = 0
        for img in a:
            idx1 = 0
            for channel_a in img:
                channel_a = channel_a.add(b[idx0][idx1]) / 2
                idx1 += 1
            idx0 += 1
    
        return a   

In [48]:
class TCNN(nn.Module):
    def __init__(self):
        super(TCNN, self).__init__()
        
        self.conv1_1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=[3,3], stride=1, padding=1)
        self.activation1 = nn.ReLU()
        
        self.pool2_B = nn.MaxPool2d(kernel_size = [4, 4], stride=2, padding=0)
        self.pool2_S = nn.MaxPool2d(kernel_size = [2, 2], stride=4, padding=0)
        
        self.conv2_B = nn.Conv2d(in_channels=8, out_channels=64, kernel_size=[4,4], stride=1)
        self.conv2_S = nn.Conv2d(in_channels=24, out_channels=64, kernel_size=[2,2], stride=1)
        self.activation2 = nn.ReLU()
        
        self.pool3_B = nn.MaxPool2d(kernel_size = [2, 2], stride=2, padding=0)
        self.pool3_S = nn.MaxPool2d(kernel_size = [4, 4], stride=4, padding=1)
        
        self.conv3_B = nn.Conv2d(in_channels=16, out_channels=128, kernel_size=[4,4], stride=1, padding=1)
        self.conv3_S = nn.Conv2d(in_channels=48, out_channels=128, kernel_size=[2,2], stride=1, padding=1)
        self.activation3 = nn.ReLU()
        
        # fc1 has 2048 inputs
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 10)
        
        self.Sigmoid = nn.Sigmoid()
        self.Softmax = nn.Softmax(dim=0)
        
        self.avg_combine = Avg_Combine()
        
    def forward(self,x):
        x = self.conv1_1(x)
        x = self.activation1(x)
          
        x_0, x_1 = torch.split(x, [8, 24], dim=1)
        x_0 = self.pool2_B(x_0)
        x_1 = self.pool2_S(x_1)
        
        x_0, x_1 = self.conv2_B(x_0), self.conv2_S(x_1)
        x = self.avg_combine(x_0, x_1) # combine 64 feature maps on both sides into one (after upsampling the small one)
        x = self.activation2(x)

        
        x_0, x_1 = torch.split(x, [16, 48], dim=1)
        x_0 = self.pool3_B(x_0)
        x_1 = self.pool3_S(x_1) 
        
        x_0, x_1 = self.conv3_B(x_0), self.conv3_S(x_1)
        x = self.avg_combine(x_0, x_1)
        x = self.activation3(x)
        x = x.view(x.size()[0], -1)
        
        x = self.fc1(x)
        x = self.Sigmoid(x)
        x = self.fc2(x)
        x = self.Sigmoid(x)
        x = self.fc3(x)
        x = self.Softmax(x)
        
        return x

In [49]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=[3,3], stride=1, padding=1)
        self.activation1 = nn.ReLU()
        
        self.pool1 = nn.MaxPool2d(kernel_size = [2, 2], stride=2, padding=0)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=[4,4], stride=1)
        self.activation2 = nn.ReLU()
        
        self.pool2 = nn.MaxPool2d(kernel_size = [2, 2], stride=2, padding=0)
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=[4,4], stride=1, padding=1)
        self.activation3 = nn.ReLU()
        
        # fc1 has 2048 inputs
        self.fc1 = nn.Linear(2048, 10)
        
        self.Sigmoid = nn.Sigmoid()
        self.Softmax = nn.Softmax(dim=0)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.activation1(x)
        
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.activation2(x)
        
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.activation3(x)
          
        x = x.view(x.size()[0], -1)
        
        x = self.fc1(x)
        x = self.Softmax(x)
        
        return x

In [62]:
def one_hot_embedding(labels, num_classes):
    """Embedding labels to one-hot form.

    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.

    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes) 
    return y[labels]

model = TCNN()

model = model.cuda()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

TCNN(
  (conv1_1): Conv2d(1, 32, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (activation1): ReLU()
  (pool2_B): MaxPool2d(kernel_size=[4, 4], stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2_S): MaxPool2d(kernel_size=[2, 2], stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2_B): Conv2d(8, 64, kernel_size=[4, 4], stride=(1, 1))
  (conv2_S): Conv2d(24, 64, kernel_size=[2, 2], stride=(1, 1))
  (activation2): ReLU()
  (pool3_B): MaxPool2d(kernel_size=[2, 2], stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool3_S): MaxPool2d(kernel_size=[4, 4], stride=4, padding=1, dilation=1, ceil_mode=False)
  (conv3_B): Conv2d(16, 128, kernel_size=[4, 4], stride=(1, 1), padding=(1, 1))
  (conv3_S): Conv2d(48, 128, kernel_size=[2, 2], stride=(1, 1), padding=(1, 1))
  (activation3): ReLU()
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)

In [63]:
optimizer = optim.Adam(params=model.parameters(), lr=0.0001, betas=(0.9, 0.999))
criterion = nn.CrossEntropyLoss()

In [64]:
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)

TCNN(
  (conv1_1): Conv2d(1, 32, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
  (activation1): ReLU()
  (pool2_B): MaxPool2d(kernel_size=[4, 4], stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2_S): MaxPool2d(kernel_size=[2, 2], stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2_B): Conv2d(8, 64, kernel_size=[4, 4], stride=(1, 1))
  (conv2_S): Conv2d(24, 64, kernel_size=[2, 2], stride=(1, 1))
  (activation2): ReLU()
  (pool3_B): MaxPool2d(kernel_size=[2, 2], stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool3_S): MaxPool2d(kernel_size=[4, 4], stride=4, padding=1, dilation=1, ceil_mode=False)
  (conv3_B): Conv2d(16, 128, kernel_size=[4, 4], stride=(1, 1), padding=(1, 1))
  (conv3_S): Conv2d(48, 128, kernel_size=[2, 2], stride=(1, 1), padding=(1, 1))
  (activation3): ReLU()
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)

In [65]:
global epoch #declear epoch global, to be used later by torch.save() 

for epoch in range(10):


    ave_loss = 0
    global loss #declear loss global, to be used later by torch.save() 
    for batch_idx, (data, target) in enumerate(train_loader):
        model.train() #set model to traning mode
        optimizer.zero_grad()
        data, target = data.float(), target.float() #set datatype
        data, target = data.to(device), target.to(device) #transfer to GPU
        data, target = Variable(data), Variable(target) #set to pytorch datatype: variable
        out = model(data) #forward pass
        loss = criterion(out, target.long()) #calculate loss
        ave_loss = ave_loss * 0.9 + loss.item() * 0.1 
        loss.backward() #back propagation with calculated loss
        optimizer.step() #calculate gradient and step
        if (batch_idx + 1) % 100 == 0 or (batch_idx + 1) == len(train_loader):
            print('==>>> epoch: {}, batch index: {}, train loss avg: {:.6f}'.format(epoch, batch_idx + 1, ave_loss))

    correct, ave_loss = 0, 0
    total_cnt = 0

    for batch_idx, (data, target) in enumerate(val_loader):
        model.eval() #set model to evaluation mode
        data, target = data.float(), target.float() #set datatype
        data, target = data.to(device), target.to(device) #transfer to GPU
        data, target = Variable(data), Variable(target) #set to pytorch datatype: variable
       
        out = model(data)
        loss = criterion(out, target.long()) #calculate loss

        pred_label = out.data
        pred_label = pred_label.argmax(dim=1, keepdim=True)
        
        target = target.long()
        target_onehot = one_hot_embedding(target.data, 10)
        target_onehot = target_onehot.to(device)
        
        total_cnt += data.data.size()[0]
        ave_loss = ave_loss * 0.9 + loss.item() * 0.1 #smooth average
        correct += pred_label.eq(target.view_as(pred_label)).sum().item()
        
        if (batch_idx + 1) % 100 == 0 or (batch_idx + 1) == len(val_loader):
            print(
            '==>>> epoch: {}, batch index: {}, test loss: {:.6f}, acc: {:.3f}'.format(
                epoch, batch_idx + 1, ave_loss, correct * 1.0 / total_cnt))

==>>> epoch: 0, batch index: 100, train loss avg: 2.283930
==>>> epoch: 0, batch index: 200, train loss avg: 2.246003
==>>> epoch: 0, batch index: 300, train loss avg: 2.231600
==>>> epoch: 0, batch index: 400, train loss avg: 2.223202
==>>> epoch: 0, batch index: 500, train loss avg: 2.220215
==>>> epoch: 0, batch index: 600, train loss avg: 2.218302
==>>> epoch: 0, batch index: 100, test loss: 2.218362, acc: 0.873
==>>> epoch: 1, batch index: 100, train loss avg: 2.217690
==>>> epoch: 1, batch index: 200, train loss avg: 2.216404
==>>> epoch: 1, batch index: 300, train loss avg: 2.216089
==>>> epoch: 1, batch index: 400, train loss avg: 2.215819
==>>> epoch: 1, batch index: 500, train loss avg: 2.215628
==>>> epoch: 1, batch index: 600, train loss avg: 2.215620
==>>> epoch: 1, batch index: 100, test loss: 2.215426, acc: 0.898
==>>> epoch: 2, batch index: 100, train loss avg: 2.215406
==>>> epoch: 2, batch index: 200, train loss avg: 2.215170
==>>> epoch: 2, batch index: 300, train lo

In [20]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

2395018
